## DAB200 -- Lab 6

In this lab, you will gain some experience in dealing with missing data and further practice converting non-numeric features in a dataset to numeric.

**Target**: to predict `Comb Unadj FE - Conventional Fuel`

### Part 0


 - **<u>Group Number:</u>** 20
 - **<u>Group Members:</u>**<br>
 Pallavi Ravikumar Menon<br>
 Akshara Santhosh


### Part 1 - Create and evaluate an initial model
#### Code (10 marks)

In [1]:
#importing libraries
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from rfpimp import *
import category_encoders as ce
import warnings
warnings.filterwarnings('ignore')

C:\Users\palla\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
#reading the data
df = pd.read_csv("C:/Users/palla/OneDrive/Desktop/ML_notebooks/Lab6/veh18_missing.csv")
df.head().T

,0,1,2,3,4
Eng Displ,NaN,6.2,2,2,NaN
# Cyl,6,8,4,4,8
Comb Unadj FE - Conventional Fuel,26.21,21.8108,35.6914,36.7994,18.9885
# Gears,6,8,8,7,6
Max Ethanol % - Gasoline,15,10,10,10,15
Intake Valves Per Cyl,2,1,2,2,2
Exhaust Valves Per Cyl,2,1,2,2,2
Fuel Metering Sys Desc,Multipoint/sequential fuel injection,Spark Ignition Direct Injection,unknown,unknown,Multipoint/sequential fuel injection
Air Aspiration Method Desc,Naturally Aspirated,Naturally Aspirated,Turbocharged,Turbocharged,Naturally Aspirated
Trans Desc,XXXXX,XXXXX,Semi-Automatic,Automated Manual,XXXXX


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1290 entries, 0 to 1289
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Eng Displ                          912 non-null    float64
 1   # Cyl                              1290 non-null   int64  
 2   Comb Unadj FE - Conventional Fuel  1290 non-null   float64
 3   # Gears                            1290 non-null   int64  
 4   Max Ethanol % - Gasoline           1252 non-null   float64
 5   Intake Valves Per Cyl              1290 non-null   int64  
 6   Exhaust Valves Per Cyl             1290 non-null   int64  
 7   Fuel Metering Sys Desc             1290 non-null   object 
 8   Air Aspiration Method Desc         1290 non-null   object 
 9   Trans Desc                         1290 non-null   object 
 10  Cyl Deact?                         1061 non-null   object 
 11  Var Valve Lift?                    1290 non-null   objec

In [4]:
#Taking initial look at the data
def sniff(df):
    with pd.option_context("display.max_colwidth", 20):
        info = pd.DataFrame()
        info['sample'] = df.iloc[0]
        info['data type'] = df.dtypes
        info['percent missing'] = df.isnull().sum()*100/len(df)
        info['Number of unique values'] = df.apply(lambda x: len(x.unique()))
        return info.sort_values('data type')

    
sniff(df).head(14)

,sample,data type,percent missing,Number of unique values
# Cyl,6,int64,0.000000,8
# Gears,6,int64,0.000000,8
Intake Valves Per Cyl,2,int64,0.000000,2
Exhaust Valves Per Cyl,2,int64,0.000000,2
Eng Displ,NaN,float64,29.302326,41
Comb Unadj FE - Conventional Fuel,26.21,float64,0.000000,1022
Max Ethanol % - Gasoline,15,float64,2.945736,4
Fuel Metering Sys Desc,Multipoint/sequential fuel injection,object,0.000000,5
Air Aspiration Method Desc,Naturally Aspirated,object,0.000000,4
Trans Desc,XXXXX,object,0.000000,8


In [5]:
#isolate all numeric features from original data set
#Selecting only numeric coloumns from the dataset
df_num = df[['Eng Displ', '# Cyl','Comb Unadj FE - Conventional Fuel','# Gears','Max Ethanol % - Gasoline','Intake Valves Per Cyl','Exhaust Valves Per Cyl']]
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1290 entries, 0 to 1289
Data columns (total 7 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Eng Displ                          912 non-null    float64
 1   # Cyl                              1290 non-null   int64  
 2   Comb Unadj FE - Conventional Fuel  1290 non-null   float64
 3   # Gears                            1290 non-null   int64  
 4   Max Ethanol % - Gasoline           1252 non-null   float64
 5   Intake Valves Per Cyl              1290 non-null   int64  
 6   Exhaust Valves Per Cyl             1290 non-null   int64  
dtypes: float64(3), int64(4)
memory usage: 70.7 KB


In [6]:
#checking the null values
print(df_num.isnull().sum())

Eng Displ                            378
# Cyl                                  0
Comb Unadj FE - Conventional Fuel      0
# Gears                                0
Max Ethanol % - Gasoline              38
Intake Valves Per Cyl                  0
Exhaust Valves Per Cyl                 0
dtype: int64


In [7]:
## Next we extract the features for creating the baseline model
X = df_num.drop('Comb Unadj FE - Conventional Fuel', axis=1)
y = df_num[['Comb Unadj FE - Conventional Fuel']]
X.head().T

,0,1,2,3,4
Eng Displ,NaN,6.2,2.0,2.0,NaN
# Cyl,6.0,8.0,4.0,4.0,8.0
# Gears,6.0,8.0,8.0,7.0,6.0
Max Ethanol % - Gasoline,15.0,10.0,10.0,10.0,15.0
Intake Valves Per Cyl,2.0,1.0,2.0,2.0,2.0
Exhaust Valves Per Cyl,2.0,1.0,2.0,2.0,2.0


In [8]:
#filling null values with 0
X = X.fillna(0)
print(X.isnull().sum())

Eng Displ                   0
# Cyl                       0
# Gears                     0
Max Ethanol % - Gasoline    0
Intake Valves Per Cyl       0
Exhaust Valves Per Cyl      0
dtype: int64


In [9]:
## Creating and evaluating the model
oob_r2_baseline = []
for i in range(10):
    rf = RandomForestRegressor(n_estimators=150, n_jobs=-1, oob_score=True)
    rf.fit(X,y)
    #Calculating Oob score for Baseline model
    oob_baseline = rf.oob_score_
    print(f"Baseline OOB score for the {i+1} trail: {oob_baseline:.4f}", end="\n")
    oob_r2_baseline.append(oob_baseline)

print('\n')
print(f"Average Baseline OOB Score: {np.mean(oob_r2_baseline):.2f} \n")

Baseline OOB score for the 1 trail: 0.6735
Baseline OOB score for the 2 trail: 0.6702
Baseline OOB score for the 3 trail: 0.6748
Baseline OOB score for the 4 trail: 0.6702
Baseline OOB score for the 5 trail: 0.6701
Baseline OOB score for the 6 trail: 0.6714
Baseline OOB score for the 7 trail: 0.6727
Baseline OOB score for the 8 trail: 0.6728
Baseline OOB score for the 9 trail: 0.6725
Baseline OOB score for the 10 trail: 0.6721


Average Baseline OOB Score: 0.67 



### Part 2 - Normalize missing values 
#### Code (15 marks)

In [10]:
#convert all representations of missing data to a single representation

from pandas.api.types import is_string_dtype, is_object_dtype
def df_normalize_strings(df):
    for col in df.columns:
        if is_string_dtype(df[col]) or is_object_dtype(df[col]):
            df[col] = df[col].str.lower()
            df[col] = df[col].fillna(np.nan) # make None -> np.nan
            df[col] = df[col].replace('unknown', np.nan)
            df[col] = df[col].replace('none', np.nan)
            df[col] = df[col].replace('XXXXX', np.nan)
df_normalize_strings(df)
sniff(df)

,sample,data type,percent missing,Number of unique values
# Cyl,6,int64,0.000000,8
# Gears,6,int64,0.000000,8
Intake Valves Per Cyl,2,int64,0.000000,2
Exhaust Valves Per Cyl,2,int64,0.000000,2
Eng Displ,NaN,float64,29.302326,41
Comb Unadj FE - Conventional Fuel,26.21,float64,0.000000,1022
Max Ethanol % - Gasoline,15,float64,2.945736,4
Fuel Metering Sys Desc,multipoint/sequential fuel injection,object,29.069767,5
Air Aspiration Method Desc,naturally aspirated,object,0.000000,4
Trans Desc,xxxxx,object,0.000000,8


#### Question (5 marks)

Note here all the different ways missing data was represented in the data.   

<font color=darkblue>

The different ways missing data was represented are as follows:
    
* 1) Some of the columns had blank values which was replaced by Nan.<br>

* 2) Some column had "unknown" given which was replaced by Nan.<br>

* 3) "XXXXX" in "Transc desc" column was changed to Nan.<br>

* 4) Other blank cells were also replced by Nan.br>

<font/>

### Part 3 - Categorical features
#### Code (25 marks)

In [11]:
fin = df.drop('Comb Unadj FE - Conventional Fuel', axis = 1)
cat = ['Fuel Metering Sys Desc','Air Aspiration Method Desc', 'Trans Desc', 'Cyl Deact?', 'Var Valve Lift?']
fin[cat] = fin[cat].fillna(0)

In [12]:
# Encode feature numerically using category encoders
for cat in fin:
    encoder = ce.OrdinalEncoder(mapping=[
    {'col': 'Fuel Metering Sys Desc', 'mapping':{0: 0, 'spark ignition direct injection': 1,'multipoint/sequential fuel injection' : 2,
                                              'spark ignition direct & ported injection': 3, 'common rail direct diesel injection' : 4,
                                              'direct diesel injection (non-common rail)' : 5}},
    {'col':'Air Aspiration Method Desc', 'mapping':{'none given': 0, 'Turbocharged': 1,'Naturally Aspirated': 2,'Supercharged': 3, 
                                                  'Turbocharged+Supercharged' : 4}},
    {'col':'Trans Desc', 'mapping':{'none given': 0, 'Multipoint/sequential fuel injection': 1,'Spark Ignition Direct Injection': 2,
                                    'Spark Ignition direct & ported injection': 3}},
    {'col':'Cyl Deact?', 'mapping': {'Y': 1, 'N': 0}},
    {'col':'Var Valve Lift?', 'mapping': {'Y': 1, 'N': 0}}
])

encoder.fit(fin)
fin = encoder.transform(fin)
sniff(fin)

,sample,data type,percent missing,Number of unique values
Fuel Metering Sys Desc,2.0,int32,0.000000,5
# Cyl,6.0,int64,0.000000,8
# Gears,6.0,int64,0.000000,8
Intake Valves Per Cyl,2.0,int64,0.000000,2
Exhaust Valves Per Cyl,2.0,int64,0.000000,2
Eng Displ,NaN,float64,29.302326,41
Max Ethanol % - Gasoline,15.0,float64,2.945736,4
Air Aspiration Method Desc,-1.0,float64,0.000000,1
Trans Desc,-1.0,float64,0.000000,1
Cyl Deact?,-1.0,float64,0.000000,1


In [13]:
fin.isnull().sum() # there are no null values.

Eng Displ                     378
# Cyl                           0
# Gears                         0
Max Ethanol % - Gasoline       38
Intake Valves Per Cyl           0
Exhaust Valves Per Cyl          0
Fuel Metering Sys Desc          0
Air Aspiration Method Desc      0
Trans Desc                      0
Cyl Deact?                      0
Var Valve Lift?                 0
dtype: int64

### Part 4 - Numeric features
#### Code (30 marks)

In [14]:
def fix_missing_num(fin, colname):
    fin[colname].fillna(fin[colname].median(), inplace=True)

In [15]:
fin['Eng Displ'].unique()

array([nan, 6.2, 2. , 4.4, 3. , 5.2, 1.6, 2.7, 4. , 3.6, 1.4, 3.5, 5.5,
       1.5, 2.5, 5.3, 1. , 5.7, 2.4, 2.2, 1.8, 6.5, 5. , 3.8, 5.6, 2.8,
       6.4, 2.3, 3.3, 6. , 1.2, 4.3, 2.9, 4.7, 8. , 6.3, 3.9, 3.2, 3.7,
       6.8, 6.6])

In [18]:
fix_missing_num(fin, 'Eng Displ')
fix_missing_num(fin, '# Cyl')
fix_missing_num(fin, '# Gears')
fix_missing_num(fin, 'Max Ethanol % - Gasoline')
fix_missing_num(fin, 'Intake Valves Per Cyl')
fix_missing_num(fin, 'Exhaust Valves Per Cyl')

In [19]:
fin['Eng Displ'].unique()

array([3. , 6.2, 2. , 4.4, 5.2, 1.6, 2.7, 4. , 3.6, 1.4, 3.5, 5.5, 1.5,
       2.5, 5.3, 1. , 5.7, 2.4, 2.2, 1.8, 6.5, 5. , 3.8, 5.6, 2.8, 6.4,
       2.3, 3.3, 6. , 1.2, 4.3, 2.9, 4.7, 8. , 6.3, 3.9, 3.2, 3.7, 6.8,
       6.6])

In [20]:
sniff(fin)

,sample,data type,percent missing,Number of unique values
Fuel Metering Sys Desc,2.0,int32,0.0,5
# Cyl,6.0,int64,0.0,8
# Gears,6.0,int64,0.0,8
Intake Valves Per Cyl,2.0,int64,0.0,2
Exhaust Valves Per Cyl,2.0,int64,0.0,2
Eng Displ,3.0,float64,0.0,40
Max Ethanol % - Gasoline,15.0,float64,0.0,3
Air Aspiration Method Desc,-1.0,float64,0.0,1
Trans Desc,-1.0,float64,0.0,1
Cyl Deact?,-1.0,float64,0.0,1


### Part 5 - Create and evaluate a final model
#### Code (10 marks)

In [21]:
## Creating and evaluating the final model
oob_final = []
for i in range(10):
    rf_final = RandomForestRegressor(n_estimators=150, n_jobs=-1, oob_score=True)
    rf_final.fit(fin,y)
    #OOB Score for final model
    oob_combined = rf_final.oob_score_
    print(f"Final model OOB score for the {i+1} trail: {oob_combined:.4f}", end="\n")
    oob_final.append(oob_combined)

print('\n')
print(f"Average Final model OOB Score: {np.mean(oob_final):.4f}\n")

Final model OOB score for the 1 trail: 0.6633
Final model OOB score for the 2 trail: 0.6690
Final model OOB score for the 3 trail: 0.6671
Final model OOB score for the 4 trail: 0.6693
Final model OOB score for the 5 trail: 0.6675
Final model OOB score for the 6 trail: 0.6683
Final model OOB score for the 7 trail: 0.6665
Final model OOB score for the 8 trail: 0.6652
Final model OOB score for the 9 trail: 0.6668
Final model OOB score for the 10 trail: 0.6725


Average Final model OOB Score: 0.6675



#### Questions (5 marks)

Provide answers to the following:
 1. calculate the percent difference between the results of Part 1 and Part 5 (make sure you are using the correct formula for percent difference) 
 2. based on the percent difference, state whether or not the results of Part 5 are an improvement over the results of Part 1

In [23]:
# Calculating the percentage difference in Part 1 and Part 2
Percentage_difference = ((oob_combined-oob_baseline) / (1-oob_baseline))*100
print( f"Percentage difference between Part1 and Part3 :{Percentage_difference:.2f} % ")

Percentage difference between Part1 and Part3 :0.11 % 


<font color=darkblue>

It can be seen looking at the percentage differece. The results have decresed and have not improved over part one.

<font/>